In [43]:
# All modules necessary to run this notebook

import pandas as pd
from itertools import combinations, chain
from sklearn.linear_model import LinearRegression
from sklearn.impute import KNNImputer
from numpy import corrcoef


In [2]:
# Parses data sources into usable dataframe, could take a while

raw_data_path = "" # fill in with path to raw data of 53 parameters for each observation
raw_sizes_path = "" # fill in w/ path to sizes of hail
tidy_data_path = "" # fill in path for where you want the tidy (but incomplete, i.e. NaN's present) data
col_names_path = "" # fill w/ path to parameter names file
clean_data_path = "" # fill w/ path to file you want to store clean and complete data in


with open(raw_data_path, "r") as f1:
    with open(raw_sizes_path, "r") as f2:
        with open(tidy_data_path, "w") as f3:
            with open(col_names_path, "r") as f4:
                ### Write Field Names ###
                col_names = [line.strip().rstrip("\n") for line in f4.readlines()]
                col_names.append("Hailstone Size")
                f3.write(",".join(col_names) + "\n")

                ### Read every 53 items, then read size and write all 54 into a output file ###
                pos = 2
                line = f1.readline().strip().rstrip("\n")
                entry_lst = [line]

                while line:                    
                    line = f1.readline().strip().rstrip("\n")
                    entry_lst.append(line)
                    pos += 1

                    if pos == 54:
                        pos = 1
                        entry_lst.append(f2.readline().strip().rstrip("\n"))
                        f3.write(",".join(entry_lst) + "\n")
                        entry_lst = []

FileNotFoundError: [Errno 2] No such file or directory: ''

In [44]:
# Load dataframe into memory

path = "/Users/joshuaelms/Desktop/github_repos/CSCI-B365/Meteorology_Modeling_Project/data/pretty_data.csv"
df = pd.read_csv(path)

df

,ML CAPE,ML CIN,ML LCL,ML LFC,ML EL,ML LI,ML hght0c,ML cap,ML b3km,ML brn,...,lrat,tei,sb_tlcl,ml_tlcl,mu_tlcl,t500,sweat,kinx,crav,Hailstone Size
0,565.886137,-2.456216,591.712340,760.740300,10016.261419,-2.475117,3057.724668,NaN,164.527269,23.507197,...,6.069679,14.624760,16.369450,14.650496,16.369450,-15.9184,237.167161,27.148344,11784.929088,1.25
1,93.557330,-61.118000,818.659297,1485.730600,4147.988929,1.094013,2878.872717,NaN,43.581811,2.961764,...,5.573791,11.207180,14.814369,11.829009,14.814369,-15.5371,196.419861,27.202330,1995.924762,1.00
2,416.713894,-0.701233,682.113493,751.489413,7419.731564,-2.174859,3043.083673,14.93536,145.276881,15.405312,...,6.338184,24.258178,15.830423,13.206279,15.830423,-16.9460,195.164206,9.331257,8136.811509,1.00
3,1110.622796,-12.420499,536.926037,989.547800,11364.753475,-4.154931,3532.140768,NaN,172.484246,22.193236,...,6.614233,19.432405,18.284842,16.262729,17.999692,-15.3177,250.757864,21.326550,31959.336376,1.50
4,1107.162497,-12.514324,536.912773,1008.662600,11386.082876,-4.102513,3583.432806,NaN,181.416062,23.363115,...,6.578394,21.032620,18.199308,16.380479,18.092846,-15.2050,264.888229,20.604840,32653.287157,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29097,0.000000,0.000000,2343.569759,NaN,2343.569759,12.494209,3528.145294,NaN,0.000000,0.000000,...,7.248768,25.792261,1.717536,-5.193819,7.683926,-15.9886,296.701531,17.164210,0.000000,1.25
29098,0.000000,0.000000,2326.323051,NaN,2326.323051,14.986276,3497.386732,NaN,0.000000,0.000000,...,7.258029,27.763669,0.607784,-7.145827,7.955932,-16.1519,303.557106,19.452570,0.000000,1.00
29099,0.000000,0.000000,2690.384769,NaN,2690.384769,14.638317,3482.939445,NaN,0.000000,0.000000,...,7.280055,25.590055,-0.462974,-9.417577,4.940292,-16.2501,232.337003,11.883790,0.000000,1.00
29100,0.000000,0.000000,2807.261593,NaN,2807.261593,16.123160,3451.467575,NaN,0.000000,0.000000,...,7.248819,25.573522,-0.424774,-11.976938,1.702041,-16.5189,161.945441,6.005967,0.000000,0.88


In [20]:
# Generate the correlation matrix for dataframe, show most strongly correlated variables to Hailstone Size

df_corr = df.corr()

df_corr["Hailstone Size"].abs().sort_values(ascending=False)

Hailstone Size    1.000000
ship              0.203719
crav              0.179000
lrat              0.161617
sweat             0.138882
MU LI             0.137036
SB LI             0.128339
ML LI             0.123399
SB CAPE           0.119698
MU CAPE           0.115961
DCAPE             0.113458
Shear 0-6 km      0.109000
ML CAPE           0.107440
ML EL             0.092197
SB EL             0.090327
stp_fixed         0.087747
MU EL             0.072912
tei               0.072430
SB LCL            0.071378
SB hght0c         0.070903
MU hght0c         0.070903
ML hght0c         0.070903
SB cap            0.068412
stp_mixed         0.068026
scp               0.063640
ebwd[0]           0.051901
pwat              0.049410
Eff SRH           0.047059
Eff Inflow        0.036898
mlmr              0.035647
ML LFC            0.033251
ML LCL            0.032367
ML b3km           0.029507
Shear 0-1 km      0.027687
mu_tlcl           0.027546
ML CIN            0.026495
MU cap            0.021769
M

In [46]:
# Handling missing values

# df.loc[df.isnull().any(axis=1)]

df_knn = KNNImputer().fit_transform(df)

df_knn


array([[ 5.65886137e+02, -2.45621613e+00,  5.91712340e+02, ...,
         2.71483440e+01,  1.17849291e+04,  1.25000000e+00],
       [ 9.35573302e+01, -6.11180002e+01,  8.18659297e+02, ...,
         2.72023300e+01,  1.99592476e+03,  1.00000000e+00],
       [ 4.16713894e+02, -7.01232700e-01,  6.82113493e+02, ...,
         9.33125700e+00,  8.13681151e+03,  1.00000000e+00],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  2.69038477e+03, ...,
         1.18837900e+01,  0.00000000e+00,  1.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  2.80726159e+03, ...,
         6.00596700e+00,  0.00000000e+00,  8.80000000e-01],
       [ 0.00000000e+00,  0.00000000e+00,  2.90264381e+03, ...,
         4.24247000e+00,  0.00000000e+00,  1.00000000e+00]])

In [85]:
# Convert from array to dataframe

df_knn_actual = pd.DataFrame(df_knn)

df_knn_actual.columns = df.columns

df_knn_actual

,ML CAPE,ML CIN,ML LCL,ML LFC,ML EL,ML LI,ML hght0c,ML cap,ML b3km,ML brn,...,lrat,tei,sb_tlcl,ml_tlcl,mu_tlcl,t500,sweat,kinx,crav,Hailstone Size
0,565.886137,-2.456216,591.712340,760.740300,10016.261419,-2.475117,3057.724668,14.031755,164.527269,23.507197,...,6.069679,14.624760,16.369450,14.650496,16.369450,-15.9184,237.167161,27.148344,11784.929088,1.25
1,93.557330,-61.118000,818.659297,1485.730600,4147.988929,1.094013,2878.872717,9.819021,43.581811,2.961764,...,5.573791,11.207180,14.814369,11.829009,14.814369,-15.5371,196.419861,27.202330,1995.924762,1.00
2,416.713894,-0.701233,682.113493,751.489413,7419.731564,-2.174859,3043.083673,14.935360,145.276881,15.405312,...,6.338184,24.258178,15.830423,13.206279,15.830423,-16.9460,195.164206,9.331257,8136.811509,1.00
3,1110.622796,-12.420499,536.926037,989.547800,11364.753475,-4.154931,3532.140768,18.580863,172.484246,22.193236,...,6.614233,19.432405,18.284842,16.262729,17.999692,-15.3177,250.757864,21.326550,31959.336376,1.50
4,1107.162497,-12.514324,536.912773,1008.662600,11386.082876,-4.102513,3583.432806,18.936401,181.416062,23.363115,...,6.578394,21.032620,18.199308,16.380479,18.092846,-15.2050,264.888229,20.604840,32653.287157,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29097,0.000000,0.000000,2343.569759,1107.279086,2343.569759,12.494209,3528.145294,11.552847,0.000000,0.000000,...,7.248768,25.792261,1.717536,-5.193819,7.683926,-15.9886,296.701531,17.164210,0.000000,1.25
29098,0.000000,0.000000,2326.323051,1107.279086,2326.323051,14.986276,3497.386732,11.339111,0.000000,0.000000,...,7.258029,27.763669,0.607784,-7.145827,7.955932,-16.1519,303.557106,19.452570,0.000000,1.00
29099,0.000000,0.000000,2690.384769,2630.432840,2690.384769,14.638317,3482.939445,16.114445,0.000000,0.000000,...,7.280055,25.590055,-0.462974,-9.417577,4.940292,-16.2501,232.337003,11.883790,0.000000,1.00
29100,0.000000,0.000000,2807.261593,2441.488395,2807.261593,16.123160,3451.467575,12.735316,0.000000,0.000000,...,7.248819,25.573522,-0.424774,-11.976938,1.702041,-16.5189,161.945441,6.005967,0.000000,0.88


In [94]:
# Generate the power set (without the empty set) for any given iterator
def power_set(iterable):
    pset = chain.from_iterable(combinations(iterable, r) for r in range(len(iterable)+1))
    return list(list(combo) for combo in pset if len(combo) > 0)


field_pset = power_set(df.columns[:20])

field_pset

[['ML CAPE'],
 ['ML CIN'],
 ['ML LCL'],
 ['ML LFC'],
 ['ML EL'],
 ['ML LI'],
 ['ML hght0c'],
 ['ML cap'],
 ['ML b3km'],
 ['ML brn'],
 ['MU CAPE'],
 ['MU CIN'],
 ['MU LCL'],
 ['MU LFC'],
 ['MU EL'],
 ['MU LI'],
 ['MU hght0c'],
 ['MU cap'],
 ['MU b3km'],
 ['MU brn'],
 ['ML CAPE', 'ML CIN'],
 ['ML CAPE', 'ML LCL'],
 ['ML CAPE', 'ML LFC'],
 ['ML CAPE', 'ML EL'],
 ['ML CAPE', 'ML LI'],
 ['ML CAPE', 'ML hght0c'],
 ['ML CAPE', 'ML cap'],
 ['ML CAPE', 'ML b3km'],
 ['ML CAPE', 'ML brn'],
 ['ML CAPE', 'MU CAPE'],
 ['ML CAPE', 'MU CIN'],
 ['ML CAPE', 'MU LCL'],
 ['ML CAPE', 'MU LFC'],
 ['ML CAPE', 'MU EL'],
 ['ML CAPE', 'MU LI'],
 ['ML CAPE', 'MU hght0c'],
 ['ML CAPE', 'MU cap'],
 ['ML CAPE', 'MU b3km'],
 ['ML CAPE', 'MU brn'],
 ['ML CIN', 'ML LCL'],
 ['ML CIN', 'ML LFC'],
 ['ML CIN', 'ML EL'],
 ['ML CIN', 'ML LI'],
 ['ML CIN', 'ML hght0c'],
 ['ML CIN', 'ML cap'],
 ['ML CIN', 'ML b3km'],
 ['ML CIN', 'ML brn'],
 ['ML CIN', 'MU CAPE'],
 ['ML CIN', 'MU CIN'],
 ['ML CIN', 'MU LCL'],
 ['ML CIN', 'MU L

In [95]:
# loop over every combination of powerset, return the best three
max = 0
for combination in field_pset:
    arr = df_knn_actual[combination].to_numpy()
    target = df_knn_actual["Hailstone Size"].to_numpy()
    obj = LinearRegression().fit(X=arr, y=target)
    coefficients = obj.coef_
    linear_combination = (df_knn_actual[combination]*coefficients).sum(axis=1)
    correlation = corrcoef(linear_combination, target)

    corr = correlation[0][1]

    if corr > max:
        max = corr
        variable_str = f'{", ".join(combination[:-1])}, and {combination[-1]}' if len(combination) > 2 else f'{combination[0]} and {combination[-1]}' if len(combination) == 2 else combination[0]
        print(f"Variable{'s' if len(combination) > 1 else ''} {variable_str} yield{'' if len(combination) > 1 else 's'} a correlation of {corr}")
    


Variable ML CAPE yields a correlation of 0.10744035377599044
Variable ML LI yields a correlation of 0.12339936047286572
Variable MU LI yields a correlation of 0.13703637286862047
Variables ML CAPE and ML b3km yield a correlation of 0.1413263618247371
Variables ML CIN and MU LI yield a correlation of 0.14201302848649372
Variables ML LFC and MU LI yield a correlation of 0.1519846876994179
Variables ML b3km and MU LI yield a correlation of 0.1622151837206652
Variables ML CAPE, ML b3km, and MU LI yield a correlation of 0.16265845602057286
Variables ML LFC, ML b3km, and MU LI yield a correlation of 0.16353333361028938
Variables ML LFC, MU EL, and MU LI yield a correlation of 0.16450920623554408
Variables ML cap, ML b3km, and MU LI yield a correlation of 0.16697909499255476
Variables ML b3km, MU EL, and MU LI yield a correlation of 0.17014375412784707
Variables ML CAPE, ML b3km, MU EL, and MU LI yield a correlation of 0.17046645625320145
Variables ML CIN, ML b3km, MU EL, and MU LI yield a co

KeyboardInterrupt: 

In [82]:
# Proof of concept for large-scale linreg brute force on BigRed

arr = df_knn_actual.iloc[:, [40]].to_numpy()
target = df_knn_actual["Hailstone Size"].to_numpy()

obj = LinearRegression().fit(X=arr, y=target)

In [83]:
coefficients = obj.coef_

linear_combination = (df_knn_actual.iloc[:, [40]]*coefficients).sum(axis=1)

correlation = corrcoef(linear_combination, target)

correlation

array([[1.        , 0.20371923],
       [0.20371923, 1.        ]])